In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
# Các danh sách lưu thuộc tính
link_list = []
dientich_list = []
wc_list = []
pn_list = []
price_list = []
huongnha_list = []
huongbancong_list =[]
mota= []
time_list = []
date_list = []
Loai_list =[]
quan_list = []

In [ ]:
# Ta sẽ cho Beautiful Soup duyệt qua lần lượt các Page (trang), mỗi trang sẽ lưu lại Link chi tiết của 20 sản phẩm được hiển thị
# Do có tất cả 503 trang nên ta sẽ duyệt qua 503 trang để lấy tổng cộng 10 050 link sản phẩm

# Lấy link đầu tiên từ mỗi phần tử 'div' với class 'item' trên các trang từ 1 đến 503
for x in range(1, 504):
    count = 0
    link = f'https://batdongsan.vn/ban-nha-ho-chi-minh/p{x}'
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml')

    product_list = soup.find_all('div', class_='image cover')

    for item in product_list:
        for link in item.find_all('a', href=True):
            link_list.append(link['href'])

In [ ]:
# Sau đó, ta sẽ mở từng link sản phẩm để trích xuất các thông tin bên trong sản phẩm

for x in link_list:
  # Mở từng link sản phẩm
  response = requests.get(x)

  # Kiểm tra nếu yêu cầu thành công
  if response.status_code == 200:
    # Biến để lưu trữ thông tin
    dientich = None
    wc = None
    pn = None
    price = None
    huongnha = None
    huongbancong = None
    description = None
    loai = None
    quan = None

    soup = BeautifulSoup(response.content, 'lxml')

    # -- Trích xuất
    # Trích xuất giá
    price = soup.find('strong', class_='price').text.strip()

    # Trích xuất các giá trị: diện tích , số phòng ngủ , số phòng WC , Các hướng của nhà , Hướng ban công
    ul_lists = soup.find_all('ul', class_='uk-list')
    for ul in ul_lists:
        li_elements = ul.find_all('li')
        for li in li_elements:
            text = li.text.strip()
            if 'Diện tích:' in text:
                dientich = text.split(':')[-1].strip()
            elif 'Phòng WC:' in text:
                wc = text.split(':')[-1].strip()
            elif 'Phòng ngủ:' in text:
                pn = text.split(':')[-1].strip()
            elif 'Hướng nhà:' in text:
                huongnha = text.split(':')[-1].strip()
            elif 'Hướng ban công:' in text:
                huongbancong = text.split(':')[-1].strip()

    # Trích xuất mô tả
    divs = soup.find_all('div', class_='content')

    for div in divs:
      if div.find('p'):
        description = div.get_text(separator='\n').strip()
        break
    mota.append(description)

    # Trích xuất date and time
    datetime = soup.find('time', class_='timeago').text
    time, date = datetime.split()

    # Trích xuất Quận, Loại nhà
    links = soup.select('ul.uk-breadcrumb li a')

    # Lấy văn bản của thẻ 'a' thứ hai
    if len(links) > 1:
      loai = links[1].text
      quan = links[3].text

    # Thêm các chuỗi tìm được vào List
    dientich_list.append(dientich)
    wc_list.append(wc)
    pn_list.append(pn)
    price_list.append(price)
    huongnha_list.append(huongnha)
    huongbancong_list.append(huongbancong)
    time_list.append(time)
    date_list.append(date)
    Loai_list.append(loai)
    quan_list.append(quan)

In [ ]:
# Tạo một DataFrame từ các List xây dựng được ở trên
data_scraping = {
    'Link' : link_list,
    'Diện tich': dientich_list,
    'Phòng WC': wc_list,
    'Phòng ngủ': pn_list,
    'Giá': price_list,
    'Hướng nhà': huongnha_list,
    'Hướng ban công': huongbancong_list,
    'Mô tả ' : mota,
    'Giờ đăng' : time_list,
    'Ngày đăng ' : date_list,
    'Quận' : quan_list,
    'Loại nhà' : Loai_list
}

df_scraping = pd.DataFrame(data_scraping)

In [ ]:
# Chuyển Dataframe thành File CSV để có thể sử dụng sau này
df_scraping.to_csv('data_scraping.csv', index=False)